In [3]:
import keras
from keras.utils import plot_model
import numpy as np 
import pandas as pd
from PIL import Image, ImageFilter
import os, random
from keras.models import Sequential
from keras.layers import    Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
import matplotlib.pyplot as plt 
%matplotlib inline

TRAIN_DIR = 'C:/Users/hp/Desktop/satellite/train/'
TEST_DIR = 'C:/Users/hp/Desktop/satellite/test/'

ROWS = 224
COLS = 224

CHANNELS = 3
#print(os.listdir(TRAIN_DIR))
#print()
#print(os.listdir(TEST_DIR))
#print()
train_images = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]
test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
#print(train_images)
#print()
#print(test_images)
#print()
def read_image(file_path):
    img = Image.open(file_path)
    im=img.convert("L")
    return im.resize((ROWS, COLS), Image.BICUBIC)

    
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype = np.uint8)
    
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        image = image.transpose(Image.ROTATE_90)
        data[i]=image.transpose(Image.FLIP_LEFT_RIGHT)
        
        if i % 1 == 0:
            print ('Processed {} of {}'.format(i, count))
            
    return data

train = prep_data(train_images)
test = prep_data(test_images)

print("Train shape: {}".format(train.shape))
print("Test shape:  {}".format(test.shape))

Processed 0 of 45
Processed 1 of 45
Processed 2 of 45
Processed 3 of 45
Processed 4 of 45
Processed 5 of 45
Processed 6 of 45
Processed 7 of 45
Processed 8 of 45
Processed 9 of 45
Processed 10 of 45
Processed 11 of 45
Processed 12 of 45
Processed 13 of 45
Processed 14 of 45
Processed 15 of 45
Processed 16 of 45
Processed 17 of 45
Processed 18 of 45
Processed 19 of 45
Processed 20 of 45
Processed 21 of 45
Processed 22 of 45
Processed 23 of 45
Processed 24 of 45
Processed 25 of 45
Processed 26 of 45
Processed 27 of 45
Processed 28 of 45
Processed 29 of 45
Processed 30 of 45
Processed 31 of 45
Processed 32 of 45
Processed 33 of 45
Processed 34 of 45
Processed 35 of 45
Processed 36 of 45
Processed 37 of 45
Processed 38 of 45
Processed 39 of 45
Processed 40 of 45
Processed 41 of 45
Processed 42 of 45
Processed 43 of 45
Processed 44 of 45
Processed 0 of 6
Processed 1 of 6
Processed 2 of 6
Processed 3 of 6
Processed 4 of 6
Processed 5 of 6
Train shape: (45, 3, 224, 224)
Test shape:  (6, 3, 22

In [4]:
labels = []
for i in train_images:
    if 'agricultural' in i:
        labels.append(0)
    elif 'forest' in i:
        labels.append(1)
    else:
        labels.append(2)
        
train = train.reshape(-1,224,224,3)
test = test.reshape(-1,224, 224, 3)
X_train = train.astype('float32')
X_test = test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = labels
X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
print(Y_train)
labels_test = []
for i in test_images:
    if 'agricultural' in i:
        labels_test.append(0)
    elif 'forest' in i:
        labels_test.append(1)
    else:
        labels_test.append(2)
Y_test = labels_test
Y_test = np.array(Y_test)
print(Y_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2]
[0 0 1 1 2 2]


In [5]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
model = VGG16(weights='imagenet', include_top=False)

In [6]:
print("Training matrix shape ", X_train.shape) #  m X features
print("Testing matrix shape  ", X_test.shape) #  m X features

Training matrix shape  (45, 224, 224, 3)
Testing matrix shape   (6, 224, 224, 3)


In [ ]:
features_x=model.predict(X_train)
print(features_x.shape)
X_train = features_x.reshape(45,25088)
print(X_train.shape)

In [14]:
features_xtest=model.predict(X_test)
print(features_xtest.shape)
X_test=features_xtest.reshape(6,25088)
print(X_test.shape)

(6, 7, 7, 512)
(6, 25088)


In [15]:
model=Sequential()

model.add(Dense(256, input_dim=25088, activation='relu',kernel_initializer='glorot_uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)
model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

model.add(Dense(128,input_dim=256,activation='sigmoid'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)

#model.add(Dense(4096,input_dim=4096,activation='sigmoid'))
#keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)

#model.add(Dense(4096,input_dim=1024,activation='relu'))
#keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)
#sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

model.add(Dense(3,input_dim=128,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# fitting the model 

hist=model.fit(X_train, Y_train, epochs=6)

Epoch 1/6
45/45 [==============================] - 1s 13ms/step - loss: 0.9780 - acc: 0.4889
Epoch 2/6
45/45 [==============================] - 0s 4ms/step - loss: 0.1872 - acc: 0.9778
Epoch 3/6
45/45 [==============================] - 0s 4ms/step - loss: 0.0849 - acc: 1.0000
Epoch 4/6
45/45 [==============================] - 0s 5ms/step - loss: 0.0505 - acc: 1.0000
Epoch 5/6
45/45 [==============================] - 0s 4ms/step - loss: 0.0404 - acc: 1.0000
Epoch 6/6
45/45 [==============================] - 0s 4ms/step - loss: 0.0293 - acc: 1.0000


In [16]:
model.save('final_model.h5')
plot_model(model, to_file='model.png')

from sklearn.metrics import log_loss
pred = model.predict(X_test, batch_size=32, verbose=1)
print(model.evaluate(X_test,Y_test))
print("Y_test", Y_test)
print("Predicted", np.array(pred))

6/6 [==============================] - 0s 13ms/step
[0.3859741985797882, 0.8333333134651184]
Y_test [0 2 1 0 1 2]
Predicted [[6.4170784e-01 8.4094994e-02 2.7419716e-01]
 [9.8299072e-04 7.3418822e-03 9.9167514e-01]
 [2.4054376e-03 3.0264056e-01 6.9495404e-01]
 [6.4809352e-01 6.0397446e-02 2.9150906e-01]
 [2.0827956e-03 8.1890064e-01 1.7901652e-01]
 [2.1066688e-02 1.3464196e-02 9.6546912e-01]]
